In [1]:
import pandas as pd 
import numpy as np
import os
import sys
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import warnings

warnings.simplefilter(action='ignore', category=FutureWarning)

DEFAULT_TODAY = (datetime.today() - timedelta(days=365)).strftime('%Y%m%d')
DEFAULT_LASTYEAR = datetime.today().strftime('%Y%m%d')

os.chdir(sys.path[0] + "\\..")
os.getcwd()

'd:\\Documents\\PycharmProject\\eastmoney_data'

In [2]:
RATING_WINDOW = 30

PREDICT_WINDOW = 7
prev_d = 5 * PREDICT_WINDOW // 7


In [13]:
rating_df = pd.read_csv("个股研报.csv")
rating_df['publishDate'] = pd.to_datetime(rating_df['publishDate']).dt.strftime('%Y-%m-%d')
rating_df.drop(['title','orgName','authorID'], axis=1)

,stockName,stockCode,orgCode,orgSName,publishDate,infoCode,column,predictNextTwoYearEps,predictNextTwoYearPe,predictNextYearEps,...,attachType,attachSize,attachPages,encodeUrl,sRatingName,sRatingCode,market,count,orgType,link
0,圆通速递,600233,10000296,西南证券,2021-08-28,AP202108271512758072,2004002002,NaN,NaN,0.71,...,0,1033,4,vKtTj+E3+/RAtlw65Lb6cEZcV/MyGJyW2gnEM2m0aTo=,买入,201.0,SHANGHAI,0,white,https://data.eastmoney.com/report/zw_industry....
1,晨光文具,603899,80000187,万联证券,2021-08-28,AP202108271512758071,2004002002,NaN,NaN,2.55,...,0,944,4,vKtTj+E3+/RAtlw65Lb6cOmkrUM1sX+RN4MVEehfBy0=,增持,104.0,SHANGHAI,18,white,https://data.eastmoney.com/report/zw_industry....
2,华旺科技,605377,10001043,国元证券,2021-08-28,AP202108271512758069,2004002002,NaN,NaN,1.92,...,0,1318,3,vKtTj+E3+/RAtlw65Lb6cDpDcW1rr7goMNggBmLzLvo=,买入,201.0,SHANGHAI,1,white,https://data.eastmoney.com/report/zw_industry....
3,晶盛机电,300316,10001043,国元证券,2021-08-28,AP202108271512758067,2004002002,NaN,NaN,1.83,...,0,1101,3,vKtTj+E3+/RAtlw65Lb6cEgM9hUiR8007AIDyKrWMrc=,买入,201.0,SHENZHEN,16,white,https://data.eastmoney.com/report/zw_industry....
4,立高食品,300973,80000028,国联证券,2021-08-28,AP202108271512758065,2004002002,NaN,NaN,3.52,...,0,1098,3,vKtTj+E3+/RAtlw65Lb6cAH3tQXjXMy9EfNkd3RbCpw=,买入,201.0,SHENZHEN,1,white,https://data.eastmoney.com/report/zw_industry....
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76737,吉比特,603444,10001055,国海证券,2023-04-03,AP202304031585034789,2004002001,26.18,18.21,21.96,...,0,469,6,QHs9WlBj4j+tObSXphUqs2dqQfm/Dzr1zR6P2xuolG0=,买入,104.0,SHANGHAI,15,white,https://data.eastmoney.com/report/zw_industry....
76738,邮储银行,601658,80000157,中泰证券,2023-04-03,AP202304031585033648,2004002001,1.13,4.27,1.02,...,0,1695,13,QHs9WlBj4j+tObSXphUqs9vh0b4BzBbsC6MqbDQe0wU=,增持,103.0,SHANGHAI,15,white,https://data.eastmoney.com/report/zw_industry....
76739,中国交建,601800,80000157,中泰证券,2023-04-03,AP202304031585033649,2004002001,1.42,7.00,1.29,...,0,848,4,QHs9WlBj4j+tObSXphUqsxNEnYTk0Ml7aEjSkmOmja4=,买入,104.0,SHANGHAI,10,white,https://data.eastmoney.com/report/zw_industry....
76740,七一二,603712,80000157,中泰证券,2023-04-03,AP202304031585033647,2004002001,1.52,20.00,1.24,...,0,582,5,QHs9WlBj4j+tObSXphUqs/NgtXiPbCpNJvZ6rLnytQc=,买入,104.0,SHANGHAI,2,white,https://data.eastmoney.com/report/zw_industry....


In [6]:
dirPath = "data/raw/"
tagDirPath = dirPath + "sohu_stock/"
data_date_lst = [] 
for filename in os.listdir(tagDirPath):
    if os.path.isdir(tagDirPath + filename):
        data_date_lst.append(datetime.strptime(filename, '%Y-%m-%d').date())

date = max(data_date_lst).strftime('%Y-%m-%d')
tag_df = pd.read_csv(tagDirPath + date + ".csv")

tag_df["stockCode"] = tag_df['stockCode'].astype("str").apply(lambda x: x.zfill(6)).to_list()
tag_df[~tag_df['sector'].isin(["央视50_","AH股","分拆预期","破净股","证金持股","上证50_","HS300_","上证180_","中证500","深成500","深证100R","标准普尔"])]
tag_df

,stockCode,stockName,sector,sectorCount,date
0,600337,美克家居,2025规划,81,2023-08-20
1,300720,海川智能,2025规划,81,2023-08-20
2,000810,创维数字,2025规划,81,2023-08-20
3,603111,康尼机电,2025规划,81,2023-08-20
4,300729,乐歌股份,2025规划,81,2023-08-20
...,...,...,...,...,...
63948,300900,广联航空,黑龙江板块,41,2023-08-20
63949,600853,龙建股份,黑龙江板块,41,2023-08-20
63950,301371,敷尔佳,黑龙江板块,41,2023-08-20
63951,688459,哈铁科技,黑龙江板块,41,2023-08-20


In [15]:
loading = True
stock_code_list = tag_df['stockCode'].to_list()

if loading:
    priceDirPath = dirPath + "price/"
    
    # Get the newest date of folders
    for filename in os.listdir(priceDirPath):
        data_date_lst.append(datetime.strptime(filename, '%Y-%m-%d').date())
    date = max(data_date_lst).strftime('%Y-%m-%d')    
    priceDirPath = dirPath + "price/" + date + "/"
    print(priceDirPath)

    # Loading price df
    price_df_list = []
    for stockCode_filename in os.listdir(priceDirPath):
        if stockCode_filename.split('.')[0] in stock_code_list:
            stock_price_df = pd.read_csv(priceDirPath + stockCode_filename, index_col=0)
        price_df_list.append(stock_price_df)
    price_df = pd.concat(price_df_list)

    price_df['日期'] = pd.to_datetime(price_df['日期'])
    # price_df = price_df[price_df['日期']>(price_df['日期'].max()-pd.Timedelta(days=365))]
    price_df["stockCode"] = price_df['stockCode'].astype("str").apply(lambda x: x.zfill(6)).to_list()
    price_df['流通股数'] = price_df['成交量'] * price_df['换手率']
    
    price_df = pd.merge(tag_df, price_df, how='outer',on='stockCode')

# price_df = price_df[price_df['stockCode'].isin(rating_df['stockCode'])]

data/raw/price/2023-08-20/


In [16]:
price_df

,stockCode,stockName,sector,sectorCount,date,日期,开盘,收盘,最高,最低,成交量,成交额,振幅,涨跌幅,涨跌额,换手率,流通股数
0,600337,美克家居,2025规划,81,2023-08-20,2021-08-20,3.59,3.58,3.61,3.52,123218.0,43744011.00,2.51,-0.28,-0.01,0.76,93645.68
1,600337,美克家居,2025规划,81,2023-08-20,2021-08-23,3.57,3.60,3.66,3.57,242678.0,87644924.00,2.51,0.56,0.02,1.49,361590.22
2,600337,美克家居,2025规划,81,2023-08-20,2021-08-24,3.59,3.59,3.62,3.57,126280.0,45355713.00,1.39,-0.28,-0.01,0.78,98498.40
3,600337,美克家居,2025规划,81,2023-08-20,2021-08-25,3.59,3.64,3.71,3.57,231862.0,84538298.00,3.90,1.39,0.05,1.42,329244.04
4,600337,美克家居,2025规划,81,2023-08-20,2021-08-26,3.64,3.65,3.69,3.61,199924.0,73127869.00,2.20,0.27,0.01,1.23,245906.52
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28101551,836826,盖世食品,食品饮料,126,2023-08-20,2023-08-14,6.28,6.29,6.36,6.12,2884.0,1801844.43,3.79,-0.79,-0.05,0.57,1643.88
28101552,836826,盖世食品,食品饮料,126,2023-08-20,2023-08-15,6.28,6.26,6.29,6.13,1535.0,948969.69,2.54,-0.48,-0.03,0.30,460.50
28101553,836826,盖世食品,食品饮料,126,2023-08-20,2023-08-16,6.08,6.10,6.30,6.07,2127.0,1316846.69,3.67,-2.56,-0.16,0.42,893.34
28101554,836826,盖世食品,食品饮料,126,2023-08-20,2023-08-17,6.10,6.18,6.21,6.01,1794.0,1092991.42,3.28,1.31,0.08,0.35,627.90


In [ ]:
# price_df.rename(columns={"开盘":'open','收盘':'close','成交量':'volume','成交额':'turnover',"最高":'high',"最低":'low','流通股数':'outstanding'})
price_df = price_df.groupby('日期','sector').agg({
        'close': lambda x: x.mean(),  
        'open': lambda x: x.mean(),
        'high': lambda x: x.mean(),  
        'low': lambda x: x.mean(),
        'volume': lambda x: sum(x),
        'turnover': lambda x: sum(x),
        'outstanding': lambda x: sum(x)
})

In [13]:
sector_list = price_df['sector'].drop_duplicates().to_list()

data_list = []
for sector in sector_list:
    print(sector)
    data = price_df[price_df['sector'].str.contains(sector)]
    print(data['sector'].drop_duplicates().to_list())
    data = data.drop(columns=["sector","sectorCount"])
    print(len(data))
    data = data.drop_duplicates()
    print(len(data))

    data = data.groupby('日期').agg({
            '收盘': lambda x: x.mean(),  
            '开盘': lambda x: x.mean(),
            '最高': lambda x: x.mean(),  
            '最低': lambda x: x.mean(),
            '成交量': lambda x: sum(x),
            '成交额': lambda x: sum(x),
            '流通股数': lambda x: sum(x)
    })
    # data.reset_index()
    data['sector'] = sector
    data.index = data.index.astype(str)

    data_list.append(data)

2025规划
['2025规划']
39348
39348
C2M概念
['C2M概念']
19389
19389
人工智能
['人工智能']
223359
221415
内贸流通
['内贸流通']
85414
84442
华为概念
['华为概念']
284814
282384
富时罗素
['富时罗素']
637423
633535
智能家居
['智能家居']
66888
66402
标准普尔
['标准普尔']
525670
522268
江西板块
['江西板块']
35909
35909
电商概念
['电商概念']
152226
150768
融资融券
['融资融券']
1472683
1463040
轻工制造
['轻工制造']
70834
69862
预盈预增
['预盈预增']
664613
659754
创业板综
['创业板综']
554925
551523
广东板块
['广东板块']
195121
193178
机械设备
['机械设备']
231983
231983
贬值受益
['贬值受益']
58741
58741
AIGC概念
['AIGC概念']
29350
29350
F5G概念
['F5G概念']
9984
9984
MiniLED
['MiniLED']
26684
26198
MSCI中国
['MSCI中国']
321737
320279
OLED
['OLED']
60807
59835
Web3.0
['Web3.0']
23126
22640
元宇宙概念
['元宇宙概念']
67371
66399
区块链
['区块链']
119937
118479
四川板块
['四川板块']
78894
77027
在线教育
['在线教育']
59801
59315
家用电器
['家用电器']
40177
39691
广电
['广电']
13599
13113
数据中心
['数据中心']
87291
87291
智慧城市
['智慧城市']
89279
89279
智能电视
['智能电视']
6310
6310
深成500
['深成500']
241472
239528
深港通
['深港通']
815371
807876
深股通
['深股通']
667951
662885
混合现实
['混合现实']
8772
8772
物联网
['物联网']
157943

In [21]:
df = pd.concat(data_list)
df = df.reset_index()

In [25]:
df.to_csv("price.csv",index=False)

In [2]:
df = pd.read_csv("price.csv")

In [3]:
def summary(df):
    print(f'data shape: {df.shape}')
    summ = pd.DataFrame(df.dtypes, columns=['data type'])
    summ['#missing'] = df.isnull().sum().values 
    summ['%missing'] = df.isnull().sum().values / len(df)* 100
    summ['#unique'] = df.nunique().values
    desc = pd.DataFrame(df.describe(include='all').transpose())
    summ['min'] = desc['min'].values
    summ['max'] = desc['max'].values
    summ['first value'] = df.loc[0].values
    summ['second value'] = df.loc[1].values
    summ['third value'] = df.loc[2].values
    
    return summ

In [4]:
summary(df)

data shape: (240502, 9)


,data type,#missing,%missing,#unique,min,max,first value,second value,third value
日期,object,0,0.0,486,NaN,NaN,2021-06-30,2021-07-01,2021-07-02
收盘,float64,0,0.0,226559,2.186364,274.21971,21.507778,21.170494,20.805802
开盘,float64,0,0.0,226651,2.184973,273.207101,21.356543,21.57037,21.096049
最高,float64,0,0.0,226859,2.233476,281.232754,21.82321,22.088642,21.516173
最低,float64,0,0.0,226283,2.161176,266.206232,21.00321,20.787778,20.368519
成交量,float64,0,0.0,238072,55967.0,1068646850.0,20557487.0,22445918.0,21983005.0
成交额,float64,0,0.0,239044,133033654.0,1537671276282.0,43331434792.0,51244549811.0,48928562197.0
流通股数,float64,0,0.0,239044,24905.53,6357879497.010005,63245554.84,81309314.44,59629094.14
sector,object,0,0.0,496,NaN,NaN,2025规划,2025规划,2025规划


In [6]:
df = df.sort_values(by=['sector', '日期'])
df['log_return'] = df.groupby('sector')['开盘'].apply(lambda x: x / x.shift(1)).apply(lambda x: pd.np.log(x))

In [7]:
df

,日期,收盘,开盘,最高,最低,成交量,成交额,流通股数,sector,log_return
0,2021-06-30,21.507778,21.356543,21.823210,21.003210,20557487.0,4.333143e+10,63245554.84,2025规划,NaN
1,2021-07-01,21.170494,21.570370,22.088642,20.787778,22445918.0,5.124455e+10,81309314.44,2025规划,0.009962
2,2021-07-02,20.805802,21.096049,21.516173,20.368519,21983005.0,4.892856e+10,59629094.14,2025规划,-0.022235
3,2021-07-05,20.987284,20.686543,21.412099,20.444198,19662713.0,4.541336e+10,52060019.52,2025规划,-0.019602
4,2021-07-06,20.693827,21.062963,21.332222,20.120123,21357001.0,5.193781e+10,61080880.80,2025规划,0.018033
...,...,...,...,...,...,...,...,...,...,...
94765,2023-06-26,9.911026,10.027949,10.151538,9.832821,6011734.0,3.111333e+09,11452118.66,黑龙江板块,-0.015880
94766,2023-06-27,10.146923,9.901282,10.208462,9.857436,5222625.0,2.650127e+09,9012274.90,黑龙江板块,-0.012712
94767,2023-06-28,10.162308,10.143077,10.241538,9.945897,4871349.0,2.618211e+09,7862786.93,黑龙江板块,0.024127
94768,2023-06-29,10.320256,10.188718,10.423590,10.096410,5276164.0,3.335239e+09,9027704.37,黑龙江板块,0.004490


In [8]:
df['date'] = pd.to_datetime(df['日期'])
df.set_index('date', inplace=True)
weekly_data = df.groupby('sector').resample('W').agg({
    '开盘': 'mean', '收盘': 'mean',
    '成交量': 'sum', '成交额': 'sum', '流通股数': 'mean',
    '最高': 'max', '最低': 'min'})

weekly_data = weekly_data.rename(columns={"开盘":'open','收盘':'close','成交量':'volume','成交额':'turnover',"最高":'high',"最低":'low','流通股数':'outstanding'})
weekly_data['log_return'] = weekly_data.groupby('sector')['close'].apply(lambda x: x / x.shift(1)).apply(lambda x: pd.np.log(x))
weekly_data['log_return'] = weekly_data.groupby('sector')['log_return'].shift(-1)

data = weekly_data


In [9]:
data

open      close       volume      turnover  \
sector date                                                          
2025规划 2021-07-04  21.340988  21.161358   64986410.0  1.435045e+11   
       2021-07-11  20.878222  21.031407  118602764.0  2.684868e+11   
       2021-07-18  21.469407  21.526395  137720806.0  3.051303e+11   
       2021-07-25  21.558173  21.709778  143267806.0  2.782908e+11   
       2021-08-01  21.362765  21.392494  162980189.0  3.195568e+11   
...                      ...        ...          ...           ...   
黑龙江板块  2023-06-04  10.281282  10.268718   27996827.0  1.591192e+10   
       2023-06-11  10.151231  10.124718   24898912.0  1.467480e+10   
       2023-06-18  10.200317  10.225086   30441759.0  1.611683e+10   
       2023-06-25  10.207436  10.168462   17475434.0  9.735867e+09   
       2023-07-02  10.122872  10.181128   27103206.0  1.511822e+10   

                    outstanding       high        low  log_return  
sector date                                                        
2025规划 2021-07-04  6.806132e+07  22.088642  20.368519   -0.006160  
       2021-07-11  7.720209e+07  21.770123  20.120123    0.023263  
       2021-07-18  1.000658e+08  22.378272  20.721852    0.008483  
       2021-07-25  1.065544e+08  22.519877  20.661481   -0.014723  
       2021-08-01  1.231568e+08  22.657654  19.812840    0.080794  
...                         ...        ...        ...         ...  
黑龙江板块  2023-06-04  9.515756e+06  10.544872  10.107692   -0.014122  
       2023-06-11  8.881393e+06  10.400769   9.921538    0.009864  
       2023-06-18  1.799033e+07  10.482368   9.994615   -0.005553  
       2023-06-25  9.990663e+06  10.397179  10.043590    0.001245  
       2023-07-02  9.782174e+06  10.513333   9.832821         NaN  

[51962 rows x 8 columns]

In [10]:
# Create lagged features
lag_periods = [1, 5, 10]  # Number of lag periods
for lag in lag_periods:
    for col in ['open', 'close', 'volume', 'turnover', 'outstanding', 'high', 'low']:
        data[f'{col}_lag_{lag}'] = data.groupby('sector')[col].shift(lag)


In [11]:
# Reset the index to separate 'sector' from the index
data.reset_index(level='sector', inplace=True)

# Create moving averages
moving_averages = [5, 10, 20]  # Moving average periods
for ma in moving_averages:
    for col in ['open', 'close', 'volume', 'turnover', 'outstanding', 'high', 'low']:
        data[f'{col}_ma_{ma}'] = data.groupby('sector')[col].rolling(window=ma).mean().reset_index(level='sector', drop=True)


In [12]:

# Calculate price change features
data['price_change'] = data.groupby('sector')['close'].diff()

# Calculate log return features
data['log_return_lag_1'] = data.groupby('sector')['log_return'].shift(1)

# Calculate relative indicators
data['return_vs_lag'] = data['log_return'] - data['log_return_lag_1']
data['return_vs_ma_5'] = data['log_return'] - data['close_ma_5']
data['return_vs_ma_10'] = data['log_return'] - data['close_ma_10']

# Drop rows with NaN due to feature engineering
data.dropna(inplace=True)

In [13]:
data

,sector,open,close,volume,turnover,outstanding,high,low,log_return,open_lag_1,...,volume_ma_20,turnover_ma_20,outstanding_ma_20,high_ma_20,low_ma_20,price_change,log_return_lag_1,return_vs_lag,return_vs_ma_5,return_vs_ma_10
date,,,,,,,,,,,,,,,,,,,,,
2021-11-14,2025规划,23.220123,23.376049,134248630.0,2.105135e+11,1.423883e+08,24.023333,22.497284,0.004838,22.973012,...,1.562645e+08,2.591877e+11,2.055488e+08,23.165228,21.380759,0.387259,0.016705,-0.011868,-22.791893,-22.482622
2021-11-21,2025规划,23.465333,23.489407,162110628.0,2.352561e+11,2.699457e+08,24.149012,22.699259,0.024005,23.220123,...,1.611207e+08,2.637753e+11,2.156430e+08,23.268247,21.497296,0.113358,0.004838,0.019168,-23.086148,-22.555852
2021-11-28,2025规划,24.007827,24.060099,140958955.0,2.343990e+11,1.465873e+08,24.665926,23.507284,0.009342,23.465333,...,1.622385e+08,2.620709e+11,2.191123e+08,23.413037,21.666654,0.570691,0.024005,-0.014664,-23.379266,-22.761015
2021-12-05,2025规划,24.184695,24.285913,148617893.0,2.477118e+11,1.373121e+08,24.897654,23.326914,-0.019209,24.007827,...,1.627834e+08,2.592000e+11,2.209746e+08,23.539006,21.796907,0.225814,0.009342,-0.028551,-23.659261,-22.987725
2021-12-12,2025规划,23.872543,23.823852,147222469.0,2.363000e+11,1.140056e+08,24.596790,23.038765,0.000362,24.184695,...,1.629811e+08,2.571004e+11,2.213472e+08,23.642852,21.915772,-0.462061,-0.019209,0.019571,-23.806702,-23.167820
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-09-11,黑龙江板块,9.652378,9.671351,27633784.0,1.756753e+10,1.493294e+07,9.897568,9.427838,-0.013696,9.519838,...,2.972417e+07,1.985359e+10,2.282779e+07,9.424223,8.774139,0.136919,0.014258,-0.027954,-9.708917,-9.478431
2022-09-18,黑龙江板块,9.631284,9.539797,23319305.0,1.504075e+10,1.712727e+07,9.895946,9.190270,-0.049291,9.652378,...,2.910893e+07,1.949464e+10,2.250492e+07,9.469748,8.862736,-0.131554,-0.013696,-0.035595,-9.715110,-9.549519
2022-09-25,黑龙江板块,9.136324,9.080973,18585709.0,1.130525e+10,5.080538e+06,9.331081,8.873784,0.001237,9.631284,...,2.931357e+07,1.958539e+10,2.172647e+07,9.513108,8.905203,-0.458824,-0.049291,0.050528,-9.501609,-9.501992


In [14]:
data = data.reset_index()
data.to_csv("featured_price.csv",index=False)